# Menyiapkan Data

## Di Balik Layar
- Proses paling makan waktu
- Sering tidak diajarkan
- Padahal "sampah masuk, sampah keluar"

## Memperoleh Data
- Dari sumber langsung: BI, BPS, World Bank, IMF
- Dari layanan penyedia data: ORBIS, Bloomberg, Quandl
- Dari library R: quandl, bls

## Memuat data dari file ke R

- Dengan library `rio`, kita bisa mengimpor ke R data dari berbagai macam format: `txt`, `csv`, `xls`, `xlsx`, `dbf`, `sav`, `dta`, `sas7.bdat`. 
- Data yang diimpor akan menjadi obyek R tipe `data.frame` 

In [ ]:
# install.packages('rio')
datakab = rio::import('indo-dapoer_data.csv')

## Mengecek Data

- Tampilkan keseluruhan data dengan mengenter nama obyek data

In [ ]:
datakab

- Namun untuk data yang besar, ini tidak banyak membantu bahkan bisa makan waktu lama
- Karenanya cek dulu besarnya data dari dimensi matriksnya.

In [ ]:
dim(datakab)

- `head` dan `tail` menampilkan baris teratas dan terbawah data 
- bisa menyebutkan jumlah baris yang diinginkan.

In [ ]:
head(datakab); tail(datakab, n = 8)

- Lima baris terakhir bukan bagian dari data. Kita bisa gunakan indeks **positif** untuk memilih baris yang **dipertahankan**.

In [ ]:
tail(datakab[1:12012,]

- atau gunakan indeks **negatif** untuk memilih baris yang **dibuang**

In [ ]:
library(magrittr)
datakab[-12013:-12017,] %>% tail

- `%>%` adalah operator pipe dari library `magrittr`, yang mengirimkan output dari fungsi di sebelah kiri untuk menjadi input/argumen dari fungsi sebelah kanan

- Subset juga bisa dibuat dengan memfilter tabel data agar hanya memberikan baris yang nilai variabelnya memenuhi kriteria yang ditetapkan 
- Kita bisa gunakan fungsi `which` untuk menghasilkan indeks baris yang memenuhi kriteria tersebut

In [ ]:
which(datakab$`Series Code` != "") %>% datakab[.,] %>% tail

- `$` digunakan untuk memilih kolom/variabel `Series Code` yang merupakan komponen dari tabel `datakab`
- nama variabel perlu diapit dengan *backtick* ``` hanya jika mengandung spasi  

- cara lain untuk memilih baris berdasar kriteria adalah menggunakan fungsi `filter` dari library `dplyr`

In [ ]:
filter(datakab, `Series Code` != "") 

- Uups, hasilnya tidak seperti yang diinginkan
- Ternyata kita belum memuat library `dplyr` sehingga fungsi `filter` yang digunakan di atas berasal dari library lain
- Jika ada lebih dari satu fungsi yang bernama sama, R akan memprioritaskan fungsi dari library yang dimuat paling akhir.

- Kita bisa memastikan fungsi `filter` yang digunakan adalah dari `dplyr`, walau kita belum memuatnya, dengan didului `dplyr::`

In [ ]:
dplyr::filter(datakab, `Series Code` != "") 

- jika sudah yakin indeks menghasilkan subset yang diinginkan, simpan subset tersebut menjadi objek 

In [ ]:
datakab = .Last.value

- nilai dari perintah yang dijalankan terakhir  bisa dimuat kembali dengan `.Last.value`

## Memilih variabel


- seperti memilih baris, kita bisa memilih variabel menggunakan indeks kolom yang terletak setelah koma di fungsi subset `[]`

In [ ]:
datakab[,c(1, 3:5, ncol(datakab))]

- indeks kolom angka bisa digantikan dengan vektor nama variabel 

In [ ]:
datakab[,c('Region Code', 'Series Name')]

- Pemilihan variabel juga bisa menggunakan `dplyr::select`

In [ ]:
library(dplyr); select(datakab, 'Region Code':'Series Code', -'Series Name', 
                       region_name = 'Region Name', contains('YR'), -contains('YR201'))

- Nama variabel yang mengandung spasi harus diapit tanda kutip 
- Pilih sejumlah variabel yang berurutan cukup sebutkan variabel di pinggir, `var_kiri:var_kanan` 
- Pilih semua variabel yang namanya memiliki kesamaan pola, baik diawali (`starts_with`), diakhiri (`ends_with`), atau mengandung (`contains`) karakter tertentu 
- Gunakan tanda minus (`-`) untuk mengecualikan variabel 
- Urutan kolom tabel baru mengikuti urutan variabel dalam `select`
- Mengganti nama variabel yang dipilih, `nama_baru = nama_lama`

## Merubah Nama Variabel

- jika hanya ingin mengganti nama sejumlah variabel, tanpa merubah struktur tabel data, gunakan `dplyr::rename`

In [ ]:
rename(datakab, region_name = 'Region Name', series_code = 'Series Code') [100:103,]

- fungsi subset `[]` bisa langsung digunakan tanpa piping  `%>%` setelah fungsi yang menghasilkan objek yang bisa di-subset

- merubah nama bisa pula dilakukan dengan meng-*assign* vektor nama baru dengan panjang sama seperti nama yang hendak digantikan

In [ ]:
names(datakab)[1:3] = names(datakab)[1:3] %>% toupper
names(datakab)

- `janitor::clean_names` merubah nama variabel agar mengikuti gaya standar 

In [ ]:
datakab = janitor::clean_names(datakab)
names(datakab)

## Menggabungkan data

- Untuk keperluan latihan penggabungan, datakab dipecah menjadi dua data dengan variabel berbeda kecuali variabel ID

In [ ]:
(data1 = datakab %>% select(region_name:series_code, contains('201')))

In [ ]:
(data2 = datakab %>% select(-contains('201')))

- gabungkan dua data tersebut menggunakan  `merge` dari base R, atau seri fungsi `_join` dari `dplyr`

In [ ]:
merge(data1, data2, by = c('region_code', 'series_code'))

- Penggabungan baris berdasar variabel ID dalam argumen `by`
- jika ada variabel bernama sama tapi tidak menjadi argumen `by`, maka di data baru akan ditambahi akhiran `.x` dan `.y`
- jika variabel id berbeda nama di kedua data, 
  
  `by = c("id1_I" = "id1_II", "id2_I" = "id2_II")`
- jika tidak ada argumen `by`, penggabungan dilakukan dengan semua variabel bernama sama

In [ ]:
dplyr::inner_join(data1, data2)

- Secara default, hasil merge hanya mempertahankan baris dengan ID yang terdapat di kedua data yang digabungkan. Hasil default merge ini ekuivalen dengan hasil dari `dplyr::inner_join`
- jika ingin semua baris dipertahankan walau ID hanya terdapat di salah satu data, maka gunakan argumen `all = TRUE`. Ini ekuivalen dengan hasil dari `dplyr::all_join`
- jika hanya ingin mempertahankan semua baris dari salah satu data 
  - data I gunakan `all.x = TRUE`, ekuivalen dengan `dplyr::left_join`
  - data II gunakan `all.y = TRUE`, ekuivalen dengan `dplyr::right_join`

## Menyimpan dan Menghapus Objek

- simpan semua objek R di memori ke file dengan perintah `save.image`
- gunakan `save` jika hanya sebagian objek yang ingin disimpan  
- simpan image dengan nama `.RData` jika ingin otomatis dimuat ke memori tiap awal menjalankan R dari direktori tersebut

In [ ]:
save(datakab,data1, data2, file = "datakab.rda")
save.image(file = ".RData")

- hapus objek R dari memori dengan `rm`

In [ ]:
rm(data1, data2)
ls()

- `ls` memberikan vektor nama semua objek yang ada di memori

In [ ]:
rm(list = ls())
load('datakab.rda')

- gunakan `load` untuk memuat semua objek dalam file image ke memori

- saat dimuat kembali dari file image, nama objek akan sama dengan saat disimpan 
- untuk bisa bebas memberikan nama baru pada objek saat dimuat kembali, simpan dengan `saveRDS` dan muat dengan `readRDS`
- kedua perintah ini hanya bisa menyimpan dan memuat satu objek

In [ ]:
saveRDS(datakab, file = "datakab.rds")
indo_dapoer = readRDS('datakab.rds')

## Merubah Nilai

- Bisa merubah nilai sel tertentu dengan menunjukkan lokasi baris dan kolomnya

In [ ]:
datakab[2,3]
datakab[2,3] = 'Morbidity Rate (%)'
datakab[2,3]

- Gunakan conditional `ifelse` untuk merubah semua baris atau kolom dengan nilai tertentu.

In [ ]:
mutate_ stringr::str_detect(names(datakab), "x200[0-9].+")] = stringr::str_replace(names(datakab), "x(200[0-9]).+", "\\1")]